# Trade with Python through MetaTrader5

In [8]:
import MetaTrader5 as mt
import pandas as pd

In [2]:
import sys
sys.path.append('D:/Quant_trade')
import requests
import numpy as np
import pandas as pd
from ta import volume, momentum, trend, volatility, add_all_ta_features, others
#import datetime
from tqdm import tqdm
import sqlalchemy
from datetime import time, datetime
from time import sleep
#from F import DumpCSV_and_MesToTele_FX, PNL_per_day, get_data_realtime, get_data_FX
from itertools import combinations
import MetaTrader5 as mt

In [6]:
def get_data_FX(symbol, timeframe, rows):
    ohlc_data = pd.DataFrame(mt.copy_rates_from_pos(symbol, timeframe, 0, rows))
    ohlc_data['time'] = ohlc_data.apply(lambda x: datetime.fromtimestamp(x["time"]), axis=1)
    ohlc_data = ohlc_data.rename(columns={'time': 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'real_volume': 'Volume'})
    ohlc_data.set_index("Date", inplace=True)
    return ohlc_data

print(pd.DataFrame(mt.copy_rates_from_pos('EURUSD', mt.TIMEFRAME_H1, 0,1000)))
#get_data_FX('EURUSD', timeframe=mt.TIMEFRAME_M15, rows=99999)

           time     open     high      low    close  tick_volume  spread  \
0    1715875200  1.08633  1.08668  1.08546  1.08559         2655       0   
1    1715878800  1.08560  1.08703  1.08544  1.08702         2471       0   
2    1715882400  1.08703  1.08757  1.08667  1.08751         1675       0   
3    1715886000  1.08750  1.08753  1.08699  1.08734         1292       0   
4    1715889600  1.08734  1.08755  1.08683  1.08706         1198       0   
..          ...      ...      ...      ...      ...          ...     ...   
995  1721052000  1.09095  1.09149  1.09053  1.09055         1953       1   
996  1721055600  1.09055  1.09087  1.09019  1.09072         2181       0   
997  1721059200  1.09073  1.09165  1.09067  1.09152         2422       0   
998  1721062800  1.09152  1.09177  1.09076  1.09171         2499       0   
999  1721066400  1.09171  1.09222  1.09100  1.09124         1293       1   

     real_volume  
0              0  
1              0  
2              0  
3          

In [ ]:
mt.initialize()

login = 84193298
password = "-8WyMrVz"
server = "MetaQuotes-Demo"

mt.login(login, password, server)

In [6]:

test = pd.DataFrame()
for i in range(10):
    try:
        test = get_data_FX('EURUSD', timeframe=mt.TIMEFRAME_H1, rows=1000)
        break
    except Exception as e:
        print(e)

test['Pos'] = 0
lst_time = [time(i, 0) for i in range(24)]
test.reset_index(inplace=True)

print(test)

                        Open     High      Low    Close  tick_volume  spread  \
Date                                                                           
2024-05-16 13:00:00  1.08872  1.08893  1.08859  1.08873          888       0   
2024-05-16 14:00:00  1.08873  1.08873  1.08842  1.08847          655       0   
2024-05-16 15:00:00  1.08847  1.08851  1.08797  1.08797          790       0   
2024-05-16 16:00:00  1.08796  1.08829  1.08747  1.08764         1478       0   
2024-05-16 17:00:00  1.08771  1.08821  1.08732  1.08746         1930       0   
...                      ...      ...      ...      ...          ...     ...   
2024-07-15 11:00:00  1.08883  1.08900  1.08859  1.08897         1164       1   
2024-07-15 12:00:00  1.08897  1.08919  1.08864  1.08917          884       0   
2024-07-15 13:00:00  1.08917  1.08957  1.08907  1.08940          736       1   
2024-07-15 14:00:00  1.08940  1.08947  1.08925  1.08936          547       0   
2024-07-15 15:00:00  1.08935  1.08959  1

In [9]:
# EUR_PHU_CCI1

PS = 'EUR_PHU_CCI1'
name = f'PS_PHU_{PS}'
path_csv_intraday = f'../Data/Total_gain_{PS}.csv'
path_csv_daily = f'../nas/{name}.csv'
token = '5882794375:AAHxM9d3jR9Mohqo8fM2rH4TbQA4uFzoE6E'
id = '-4047142092'
position = 0.01

############################################################################################
params = {'a': 200, 'b': 81, 'c': 16, 'd': 143, 'e': 124, 'f': 193, 'g': 15, 'h': 18}
a = params['a']
b = params['b']
c = params['c']
d = params['d']
e = params['e']
f = params['f']*0.1
g = params['g']
h = params['h']

drop_time = lst_time[g:h]

window1 = a
window2 = b
window_std = c
CCI_long = d
CCI_short = e
std_threshold1 = f

test['signal_long'] = np.nan
test['Close_MA'] = test['Close'].rolling(5).mean()
test['std'] = test['Close'].rolling(window_std).std()
test[f'CCI{window1}'] = trend.CCIIndicator(test['High'], test['Low'], test['Close_MA'], window1).cci()
test[f'CCI{window2}'] = trend.CCIIndicator(test['High'], test['Low'], test['Close_MA'], window2).cci()

test.loc[(test[f'CCI{window1}']>CCI_long), 'signal_long'] = 1
test.loc[(test[f'CCI{window2}']<=CCI_short)&(test['std'] > std_threshold1), 'signal_long'] = 0
test.loc[~test['Date'].dt.time.isin(drop_time), 'signal_long'] = 0
test['signal_long'] = test['signal_long'].ffill()
test['Pos'] += test['signal_long']

############################################################################################
params = {'a': 88, 'b': 186, 'c': 3, 'd': 193, 'e': 41, 'f': 142, 'g': 6, 'h': 12}
a = params['a']
b = params['b']
c = params['c']
d = params['d']
e = params['e']
f = params['f']*0.1
g = params['g']
h = params['h']

drop_time = lst_time[g:h]

window1 = a
window2 = b
window_std = c
CCI_long = d
CCI_short = e
std_threshold1 = f

test['signal_long'] = np.nan
test['Close_MA'] = test['Close'].rolling(5).mean()
test['std'] = test['Close'].rolling(window_std).std()
test[f'CCI{window1}'] = trend.CCIIndicator(test['High'], test['Low'], test['Close_MA'], window1).cci()
test[f'CCI{window2}'] = trend.CCIIndicator(test['High'], test['Low'], test['Close_MA'], window2).cci()

test.loc[(test[f'CCI{window1}']>CCI_long), 'signal_long'] = 1
test.loc[(test[f'CCI{window2}']<=CCI_short)&(test['std'] > std_threshold1), 'signal_long'] = 0
test.loc[test['Date'].dt.time.isin(drop_time), 'signal_long'] = 0
test['signal_long'] = test['signal_long'].ffill()
test['Pos'] += test['signal_long']

############################################################################################
params = {'a': 29, 'b': 98, 'c': 100, 'd': -167, 'e': -155, 'f': 159, 'g': 1, 'h': 2}
a = params['a']
b = params['b']
c = params['c']
d = params['d']
e = params['e']
f = params['f']*0.1
g = params['g']
h = params['h']

drop_time = lst_time[g:h]

window1 = a
window2 = b
window_std = c
CCI_long = d
CCI_short = e
std_threshold1 = f

test['signal_long'] = np.nan
test['Close_MA'] = test['Close'].rolling(5).mean()
test['std'] = test['Close'].rolling(window_std).std()
test[f'CCI{window1}'] = trend.CCIIndicator(test['High'], test['Low'], test['Close_MA'], window1).cci()
test[f'CCI{window2}'] = trend.CCIIndicator(test['High'], test['Low'], test['Close_MA'], window2).cci()

test.loc[(test[f'CCI{window1}']<CCI_long), 'signal_long'] = -1
test.loc[(test[f'CCI{window2}']>=CCI_short)&(test['std'] > std_threshold1), 'signal_long'] = 0
test.loc[test['Date'].dt.time.isin(drop_time), 'signal_long'] = 0
test['signal_long'] = test['signal_long'].ffill()
test['Pos'] += test['signal_long']

test.loc[test['Pos'] > 0, 'Pos'] = 1
test.loc[test['Pos'] < 0, 'Pos'] = -1

test
# test_update = get_data_FX('BTCUSD', timeframe=mt.TIMEFRAME_H1, rows=1000)
# df_csv, inputPos, CP = DumpCSV_and_MesToTele_FX(name, 'BTCUSD', path_csv_intraday, test['Pos'], test_update['Close'], token, id, position, fee=20)
# df_daily = PNL_per_day(path_csv_daily, df_csv['profit_today'])

,Date,Open,High,Low,Close,tick_volume,spread,Volume,Pos,signal_long,Close_MA,std,CCI200,CCI81,CCI88,CCI186,CCI29,CCI98
0,2024-05-16 13:00:00,1.08872,1.08893,1.08859,1.08873,888,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-05-16 14:00:00,1.08873,1.08873,1.08842,1.08847,655,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-05-16 15:00:00,1.08847,1.08851,1.08797,1.08797,790,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-05-16 16:00:00,1.08796,1.08829,1.08747,1.08764,1478,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-05-16 17:00:00,1.08771,1.08821,1.08732,1.08746,1930,0,0,NaN,NaN,1.088054,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2024-07-15 11:00:00,1.08883,1.08900,1.08859,1.08897,1164,1,0,0.0,0.0,1.088808,0.003041,161.541981,88.837585,94.692674,169.299417,21.788698,103.897247
996,2024-07-15 12:00:00,1.08897,1.08919,1.08864,1.08917,884,0,0,0.0,0.0,1.088846,0.003066,163.129226,88.742050,94.314345,168.975125,23.383898,103.300970
997,2024-07-15 13:00:00,1.08917,1.08957,1.08907,1.08940,736,1,0,0.0,0.0,1.088986,0.003097,169.926448,93.988684,99.183250,174.906961,37.785861,108.009931
998,2024-07-15 14:00:00,1.08940,1.08947,1.08925,1.08936,547,0,0,0.0,0.0,1.089144,0.003123,170.476328,93.589815,98.509176,173.992887,39.209848,106.970408
